In [0]:
## Cat et al 2015 Paper
## Doi : https://doi.org/10.1016/j.asoc.2015.01.025

In [0]:
## Libraries Prequisities
!pip install scikit-learn
!pip install numpy==1.16.1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/")
!ls

'My Drive'


In [0]:
import os
os.chdir("My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/")

In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

In [0]:
## Catal_2015 Paper

In [0]:
def RunOpportunity():
  X_train,Y_train,X_test,Y_test=OpportunityDataSetAnalysis()
  TrainAndTestOpportunity(X_train,Y_train,X_test,Y_test)

In [0]:
def DataSetAnalysis(data_input_file):
  data = np.load(data_input_file)
  # DataSet of 2555 Rows
  # Each Row consists of 3 columns
  ## X, y, folds
  ## For X set -> 2555 rows
  ##  Each row has 5750 elements
  X = data['X']
  print("Length of X: ",len(X))
  print("Size of X[0]: ",X[0].size)
  print("X Shape before reshape: ",X.shape)
  X = X[:, 0, :, :]
  print("X Shape after reshape: ",X.shape)
  ## For Y set -> 2555 rows
  ## Each row has 12 elements
  ## Since we have 12 classes so it is hot vector of 12
  Y = data['y']
  print("Length of Y: ", len(Y))
  print("First Element Y size: ",Y[0].size)
  print("First Element Size: " ,Y[0])
  ## Folds 2D array that contains
  ##    1- Number of Folds for cross Validation
  ##    2- Data Size for Train + Test
  ##    3- Folds[][0]-> Data for Train
  ##    4- Folds[][1] -> Data for Test
  ##    
  folds = data['folds']
  print("Length of Folds: ",len(folds))
  print("Sample of Train Data: ",folds[1][0])
  print("Sample of Test Data: ",folds[1][1])
  print("Length Sample of Train Data: ",len(folds[1][0]))
  print("Length Sample of Test Data: ",len(folds[1][1]))
  classes_number = Y.shape[1]
  print("Number of Classes: ", classes_number)
  train_idx = folds[0][0]
  test_idx = folds[0][1]
  X_train = feature_extraction(X[train_idx])
  print('Train X element 0: ', X_train[0])
  print("Train X Index shape: ", X_train.shape)
  print("Train Y Index shape: ", Y[train_idx].shape)
  

In [0]:
def TrainDataAnalysis():
  train_idx = folds[0][0]
  test_idx = folds[0][1]

  print("Train Index shape: ", train_idx.shape)
  print("Test Index shape: ", test_idx.shape)

  X_train = X[train_idx]
  X_test = X[test_idx]

  print("Train shape before Feature Extraction: ", X_train.shape)
  print("Test shape before Feature Extraction: ", X_test.shape)

  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)

  print("Train shape after Feature Extraction: ", X_train.shape)
  print("Test shape after Feature Extraction: ", X_test.shape)

In [0]:
DataSetAnalysis('data/LOSO/USCHAD.npz')

Length of X:  9824
Size of X[0]:  3000
X Shape before reshape:  (9824, 1, 500, 6)
X Shape after reshape:  (9824, 500, 6)
Length of Y:  9824
First Element Y size:  12
First Element Size:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Length of Folds:  14
Sample of Train Data:  [   0    1    2 ... 9821 9822 9823]
Sample of Test Data:  [  90   91   92   93   94   95   96   97   98   99  100  101  102  103
  104  105  106  107  108  109  110  111  112  113  114  115  116  117
  118  119  120  121  122  123  124  125  126  127  128  129  130  131
  132  133  134  135  136  137  138  139  140  141  142  143  144  145
  146  147  148  149  150  151  152  153  154  155  156  157  158  159
  160  161  162  163  164  165  166  167  168  169  170  171  172  173
  174  175  176  177  178  179 1411 1412 1413 1414 1415 1416 1417 1418
 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428 1429 1430 1431 1432
 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442 1443 1444 1445 1446
 1447 1448 1449 1450 1451 1452 145

In [0]:
TrainDataAnalysis()

Train Index shape:  (2300,)
Test Index shape:  (255,)
Train shape before Feature Extraction:  (2300, 250, 23)
Test shape before Feature Extraction:  (255, 250, 23)
Train shape after Feature Extraction:  (2300, 93)
Test shape after Feature Extraction:  (255, 93)


In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks)

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        #features = np.hstack((features, TBP(sample)))
        X_tmp.append(features)

    X = np.array(X_tmp)
    return X

In [0]:
# Classical Machine Learning Algos
def train_j48(X, y):
    from sklearn import tree
    clf = tree.DecisionTreeClassifier()
    #clf = clf.fit(X, y)
    return clf

def train_mlp(X, y):
    from sklearn.neural_network import MLPClassifier
    a = int((X.shape[1] + np.amax(y)) / 2 )#Default param of weka, amax(y) gets the number of classes
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes = (a,),
                        learning_rate_init=0.3, momentum=0.2, max_iter=500, #Default param of weka
                        )
    #clf.fit(X, y)
    return clf

def train_logistic_regression(X, y):
    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(multi_class='ovr')
    #clf.fit(X, y)
    return clf

In [0]:
def DataPreparation(data_input_file):
  print('Catal et al. 2015 {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [0]:
def Train(X,Y,folds):
  avg_acc = []
  avg_recall = []
  avg_f1 = []
  for i in range(0, len(folds)):
          train_idx = folds[i][0]
          test_idx = folds[i][1]

          X_train = X[train_idx]
          X_test = X[test_idx]

          X_train = feature_extraction(X_train)
          X_test = feature_extraction(X_test)

          j_48 = train_j48(X_train,Y[train_idx])
          mlp = train_mlp(X_train, Y[train_idx])
          logistic_regression = train_logistic_regression(X_train, Y[train_idx])

          majority_voting = VotingClassifier(estimators=[('dt', j_48), ('mlp', mlp), ('lr', logistic_regression)], voting='soft')
          majority_voting.fit(X_train, Y[train_idx])
          tmp = majority_voting.predict(X_test)

          acc_fold = accuracy_score(Y[test_idx], tmp)
          avg_acc.append(acc_fold)

          recall_fold = recall_score(Y[test_idx], tmp, average='macro')
          avg_recall.append(recall_fold)

          f1_fold  = f1_score(Y[test_idx], tmp, average='macro')
          avg_f1.append(f1_fold)

          print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
          print('________________________________________________________________')
    
  return avg_acc, avg_recall,avg_f1    

In [0]:
def ReportAccuracies(avg_acc, avg_recall,avg_f1):
  ic_acc = st.t.interval(0.9, len(avg_acc) - 1, loc=np.mean(avg_acc), scale=st.sem(avg_acc))
  ic_recall = st.t.interval(0.9, len(avg_recall) - 1, loc=np.mean(avg_recall), scale=st.sem(avg_recall))
  ic_f1 = st.t.interval(0.9, len(avg_f1) - 1, loc=np.mean(avg_f1), scale=st.sem(avg_f1))
  print('Mean Accuracy[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_acc), ic_acc[0], ic_acc[1]))
  print('Mean Recall[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_recall), ic_recall[0], ic_recall[1]))
  print('Mean F1[{:.4f}] IC [{:.4f}, {:.4f}]'.format(np.mean(avg_f1), ic_f1[0], ic_f1[1]))

In [0]:
def OpportunityDataSetAnalysis():
  X_train=np.load('data/Opportunity/Opportunity_train_X.npz')['arr_0']
  X_test=np.load('data/Opportunity/Opportunity_test_X.npz')['arr_0']
  Y_train=np.load('data/Opportunity/Opportunity_train_y.npz')['arr_0']
  Y_test=np.load('data/Opportunity/Opportunity_test_Y.npz')['arr_0']
  Y_train = np.argmax(Y_train, axis=1)
  Y_test = np.argmax(Y_test, axis=1)
  return X_train,Y_train,X_test,Y_test

In [0]:
def UCIHeterognityDataSetAnalysis():
  XDataSet=np.load('data/UCI_Heterognity/UCI_HHAR_X_2.npz')['arr_0']
  YDataSet=np.load('data/UCI_Heterognity/UCI_HHAR_Y_2.npz')['arr_0']
  XDataSet = np.argmax(XDataSet, axis=1)
  YDataSet = np.argmax(YDataSet, axis=1)
  from sklearn.model_selection import train_test_split
  X_train,Y_train, X_test, Y_test = train_test_split(XDataSet, YDataSet, test_size=0.2, random_state=42)
  return X_train,Y_train,X_test,Y_test

In [0]:
def UCIHeterognityDataSetAnalysisNonOverlapping():
  XDataSet=np.load('data/UCI_Heterognity/UCI_HHAR_X.npz')['arr_0']
  YDataSet=np.load('data/UCI_Heterognity/UCI_HHAR_Y.npz')['arr_0']
  XDataSet = np.argmax(XDataSet, axis=1)
  YDataSet = np.argmax(YDataSet, axis=1)
  from sklearn.model_selection import train_test_split
  X_train,Y_train, X_test, Y_test = train_test_split(XDataSet, YDataSet, test_size=0.2, random_state=42)
  return X_train,Y_train,X_test,Y_test

In [0]:
def RunCatal(data_input_file):
  X,Y,folds= DataPreparation(data_input_file)
  avg_acc, avg_recall,avg_f1= Train(X,Y,folds)
  ReportAccuracies(avg_acc, avg_recall,avg_f1 )

In [0]:
def TrainAndTestOpportunity(X_train,Y_train,X_test,Y_test):
    X_train = feature_extraction(X_train)
    X_test = feature_extraction(X_test)

    j_48 = train_j48(X_train,Y_train)
    mlp = train_mlp(X_train, Y_train)
    logistic_regression = train_logistic_regression(X_train, Y_train)

    majority_voting = VotingClassifier(estimators=[('dt', j_48), ('mlp', mlp), ('lr', logistic_regression)], voting='soft')
    majority_voting.fit(X_train, Y_train)
    tmp = majority_voting.predict(X_test)

    acc = accuracy_score(Y_test, tmp)

    recall = recall_score(Y_test, tmp, average='macro')

    f1 = f1_score(Y_test, tmp, average='macro')

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}]'.format(acc, recall, f1))
    print('________________________________________________________________') 

In [0]:
def TrainAndTestUCI(X_train,Y_train,X_test,Y_test):
    j_48 = train_j48(X_train,Y_train)
    mlp = train_mlp(X_train, Y_train)
    logistic_regression = train_logistic_regression(X_train, Y_train)

    majority_voting = VotingClassifier(estimators=[('dt', j_48), ('mlp', mlp), ('lr', logistic_regression)], voting='soft')
    majority_voting.fit(X_train, Y_train)
    tmp = majority_voting.predict(X_test)

    acc = accuracy_score(Y_test, tmp)

    recall = recall_score(Y_test, tmp, average='macro')

    f1 = f1_score(Y_test, tmp, average='macro')

    print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}]'.format(acc, recall, f1))
    print('________________________________________________________________') 

In [0]:
def RunUCIHeterognity():
  X_train, X_test, Y_train, Y_test=UCIHeterognityDataSetAnalysis()
  print(X_train.shape,Y_train.shape)
  print(X_test.shape,Y_test.shape)
  TrainAndTestUCI(X_train,Y_train,X_test,Y_test)

In [0]:
def RunUCIHeterognityNonOverlapping():
  X_train, X_test, Y_train, Y_test=UCIHeterognityDataSetAnalysisNonOverlapping()
  print(X_train.shape,Y_train.shape)
  print(X_test.shape,Y_test.shape)
  TrainAndTestUCI(X_train,Y_train,X_test,Y_test)

In [15]:
RunUCIHeterognityNonOverlapping()

(1044997, 6) (1044997,)
(261250, 6) (261250,)
Accuracy[0.2813] Recall[0.1758] F1[0.0988]
________________________________________________________________


In [25]:
RunUCIHeterognity()

(81640, 6) (81640,)
(20410, 6) (20410,)
Accuracy[0.2369] Recall[0.2217] F1[0.2219]
________________________________________________________________


In [33]:
RunOpportunity()

Accuracy[0.8545] Recall[0.8680] F1[0.8711]
________________________________________________________________


In [0]:
RunCatal('data/LOSO/MHEALTH.npz')

Catal et al. 2015 data/LOSO/MHEALTH.npz
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[1]
________________________________________________________________
Accuracy[0.9125] Recall[0.9246] F1[0.9091] at fold[2]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[3]
________________________________________________________________
Accuracy[0.9098] Recall[0.8965] F1[0.8797] at fold[4]
________________________________________________________________
Accuracy[0.8340] Recall[0.8406] F1[0.7928] at fold[5]
________________________________________________________________
Accuracy[0.9167] Recall[0.9203] F1[0.9003] at fold[6]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________
Accuracy

In [0]:
RunCatal('data/LOSO/USCHAD.npz')

Catal et al. 2015 data/LOSO/USCHAD.npz
Accuracy[0.7525] Recall[0.7304] F1[0.6953] at fold[1]
________________________________________________________________
Accuracy[0.6027] Recall[0.5504] F1[0.5162] at fold[2]
________________________________________________________________
Accuracy[0.7836] Recall[0.7377] F1[0.7049] at fold[3]
________________________________________________________________
Accuracy[0.7627] Recall[0.6674] F1[0.6496] at fold[4]
________________________________________________________________
Accuracy[0.5467] Recall[0.4629] F1[0.4470] at fold[5]
________________________________________________________________
Accuracy[0.7799] Recall[0.7570] F1[0.7350] at fold[6]
________________________________________________________________
Accuracy[0.6214] Recall[0.6286] F1[0.5752] at fold[7]
________________________________________________________________
Accuracy[0.8555] Recall[0.8086] F1[0.7961] at fold[8]
________________________________________________________________
Accuracy[

In [0]:
RunCatal('data/LOSO/UTD-MHAD1_1s.npz')

Catal et al. 2015 data/LOSO/UTD-MHAD1_1s.npz
Accuracy[0.3531] Recall[0.3455] F1[0.3435] at fold[1]
________________________________________________________________
Accuracy[0.3333] Recall[0.3354] F1[0.3152] at fold[2]
________________________________________________________________
Accuracy[0.2867] Recall[0.2768] F1[0.2752] at fold[3]
________________________________________________________________
Accuracy[0.3141] Recall[0.2985] F1[0.2986] at fold[4]
________________________________________________________________
Accuracy[0.2991] Recall[0.3017] F1[0.2955] at fold[5]
________________________________________________________________
Accuracy[0.3471] Recall[0.3467] F1[0.3331] at fold[6]
________________________________________________________________
Accuracy[0.3403] Recall[0.3305] F1[0.3287] at fold[7]
________________________________________________________________
Accuracy[0.2844] Recall[0.2866] F1[0.2717] at fold[8]
________________________________________________________________
Mea

In [0]:
RunCatal('data/LOSO/UTD-MHAD2_1s.npz')

Catal et al. 2015 data/LOSO/UTD-MHAD2_1s.npz
Accuracy[0.8333] Recall[0.8321] F1[0.8330] at fold[1]
________________________________________________________________
Accuracy[0.8182] Recall[0.8018] F1[0.8085] at fold[2]
________________________________________________________________
Accuracy[0.7237] Recall[0.7483] F1[0.7104] at fold[3]
________________________________________________________________
Accuracy[0.8699] Recall[0.8723] F1[0.8646] at fold[4]
________________________________________________________________
Accuracy[0.7320] Recall[0.7153] F1[0.7219] at fold[5]
________________________________________________________________
Accuracy[0.7159] Recall[0.7015] F1[0.6982] at fold[6]
________________________________________________________________
Accuracy[0.7857] Recall[0.7886] F1[0.7911] at fold[7]
________________________________________________________________
Accuracy[0.4150] Recall[0.3894] F1[0.3740] at fold[8]
________________________________________________________________
Mea

In [0]:
RunCatal('data/LOSO/WHARF.npz')

Catal et al. 2015 data/LOSO/WHARF.npz
Accuracy[0.5030] Recall[0.4212] F1[0.4123] at fold[1]
________________________________________________________________
Accuracy[0.4439] Recall[0.2367] F1[0.2388] at fold[2]
________________________________________________________________
Accuracy[0.4087] Recall[0.2942] F1[0.2706] at fold[3]
________________________________________________________________
Accuracy[0.4435] Recall[0.2938] F1[0.2636] at fold[4]
________________________________________________________________
Accuracy[0.8571] Recall[0.7000] F1[0.7222] at fold[5]
________________________________________________________________
Accuracy[0.6014] Recall[0.5100] F1[0.4941] at fold[6]
________________________________________________________________
Accuracy[0.5126] Recall[0.5347] F1[0.4858] at fold[7]
________________________________________________________________
Accuracy[0.4196] Recall[0.2362] F1[0.2334] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/LOSO/WISDM.npz')

Catal et al. 2015 data/LOSO/WISDM.npz
Accuracy[0.7850] Recall[0.6134] F1[0.5856] at fold[1]
________________________________________________________________
Accuracy[0.3648] Recall[0.1933] F1[0.2033] at fold[2]
________________________________________________________________
Accuracy[0.7559] Recall[0.7129] F1[0.7091] at fold[3]
________________________________________________________________
Accuracy[0.8791] Recall[0.8399] F1[0.8391] at fold[4]
________________________________________________________________
Accuracy[0.7965] Recall[0.6228] F1[0.6223] at fold[5]
________________________________________________________________
Accuracy[0.9298] Recall[0.6449] F1[0.6784] at fold[6]
________________________________________________________________
Accuracy[0.6915] Recall[0.5792] F1[0.5702] at fold[7]
________________________________________________________________
Accuracy[0.3420] Recall[0.3316] F1[0.3325] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/SNOW/MHEALTH.npz')


Catal et al. 2015 data/SNOW/MHEALTH.npz
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[1]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[2]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[3]
________________________________________________________________
Accuracy[0.9961] Recall[0.9964] F1[0.9964] at fold[4]
________________________________________________________________
Accuracy[0.9961] Recall[0.9964] F1[0.9964] at fold[5]
________________________________________________________________
Accuracy[0.9961] Recall[0.9964] F1[0.9964] at fold[6]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________
Accuracy

In [0]:
RunCatal('data/SNOW/USCHAD.npz')


Catal et al. 2015 data/SNOW/USCHAD.npz
Accuracy[0.9170] Recall[0.8915] F1[0.8907] at fold[1]
________________________________________________________________
Accuracy[0.8978] Recall[0.8689] F1[0.8737] at fold[2]
________________________________________________________________
Accuracy[0.8976] Recall[0.8684] F1[0.8704] at fold[3]
________________________________________________________________
Accuracy[0.9046] Recall[0.8862] F1[0.8869] at fold[4]
________________________________________________________________
Accuracy[0.9266] Recall[0.9017] F1[0.9035] at fold[5]
________________________________________________________________
Accuracy[0.9194] Recall[0.8967] F1[0.8978] at fold[6]
________________________________________________________________
Accuracy[0.9081] Recall[0.8809] F1[0.8844] at fold[7]
________________________________________________________________
Accuracy[0.9142] Recall[0.8859] F1[0.8880] at fold[8]
________________________________________________________________
Accuracy[

In [0]:
RunCatal('data/SNOW/UTD-MHAD1_1s.npz')


Catal et al. 2015 data/SNOW/UTD-MHAD1_1s.npz
Accuracy[0.5377] Recall[0.5384] F1[0.5321] at fold[1]
________________________________________________________________
Accuracy[0.4503] Recall[0.4487] F1[0.4505] at fold[2]
________________________________________________________________
Accuracy[0.5171] Recall[0.5101] F1[0.5045] at fold[3]
________________________________________________________________
Accuracy[0.4777] Recall[0.4752] F1[0.4716] at fold[4]
________________________________________________________________
Accuracy[0.5316] Recall[0.5241] F1[0.5170] at fold[5]
________________________________________________________________
Accuracy[0.4828] Recall[0.4818] F1[0.4828] at fold[6]
________________________________________________________________
Accuracy[0.4947] Recall[0.4933] F1[0.4850] at fold[7]
________________________________________________________________
Accuracy[0.4718] Recall[0.4699] F1[0.4629] at fold[8]
________________________________________________________________
Acc

In [0]:
RunCatal('data/SNOW/UTD-MHAD2_1s.npz')


Catal et al. 2015 data/SNOW/UTD-MHAD2_1s.npz
Accuracy[0.7542] Recall[0.7517] F1[0.7479] at fold[1]
________________________________________________________________
Accuracy[0.7458] Recall[0.7570] F1[0.7435] at fold[2]
________________________________________________________________
Accuracy[0.8017] Recall[0.7855] F1[0.7944] at fold[3]
________________________________________________________________
Accuracy[0.8496] Recall[0.8495] F1[0.8439] at fold[4]
________________________________________________________________
Accuracy[0.8482] Recall[0.8464] F1[0.8418] at fold[5]
________________________________________________________________
Accuracy[0.7946] Recall[0.7902] F1[0.7891] at fold[6]
________________________________________________________________
Accuracy[0.8750] Recall[0.8672] F1[0.8729] at fold[7]
________________________________________________________________
Accuracy[0.8304] Recall[0.8227] F1[0.8201] at fold[8]
________________________________________________________________
Acc

In [0]:
RunCatal('data/SNOW/WHARF.npz')

Catal et al. 2015 data/SNOW/WHARF.npz
Accuracy[0.6759] Recall[0.6007] F1[0.5934] at fold[1]
________________________________________________________________
Accuracy[0.6624] Recall[0.5711] F1[0.5737] at fold[2]
________________________________________________________________
Accuracy[0.6650] Recall[0.5948] F1[0.6009] at fold[3]
________________________________________________________________
Accuracy[0.6658] Recall[0.5561] F1[0.5565] at fold[4]
________________________________________________________________
Accuracy[0.6838] Recall[0.6186] F1[0.6195] at fold[5]
________________________________________________________________
Accuracy[0.6382] Recall[0.5780] F1[0.5730] at fold[6]
________________________________________________________________
Accuracy[0.6537] Recall[0.5935] F1[0.5793] at fold[7]
________________________________________________________________
Accuracy[0.6494] Recall[0.5819] F1[0.5698] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/SNOW/WISDM.npz')

Catal et al. 2015 data/SNOW/WISDM.npz
Accuracy[0.9076] Recall[0.8463] F1[0.8684] at fold[1]
________________________________________________________________
Accuracy[0.9018] Recall[0.8430] F1[0.8668] at fold[2]
________________________________________________________________
Accuracy[0.8974] Recall[0.8372] F1[0.8590] at fold[3]
________________________________________________________________
Accuracy[0.9046] Recall[0.8488] F1[0.8686] at fold[4]
________________________________________________________________
Accuracy[0.9155] Recall[0.8675] F1[0.8875] at fold[5]
________________________________________________________________
Accuracy[0.9030] Recall[0.8555] F1[0.8712] at fold[6]
________________________________________________________________
Accuracy[0.9126] Recall[0.8607] F1[0.8797] at fold[7]
________________________________________________________________
Accuracy[0.9069] Recall[0.8524] F1[0.8692] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/LOTO/MHEALTH.npz')


Catal et al. 2015 data/LOTO/MHEALTH.npz
Accuracy[0.8828] Recall[0.8913] F1[0.8856] at fold[1]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[2]
________________________________________________________________
Accuracy[0.7873] Recall[0.8023] F1[0.7580] at fold[3]
________________________________________________________________
Accuracy[0.9608] Recall[0.9638] F1[0.9626] at fold[4]
________________________________________________________________
Accuracy[0.9163] Recall[0.9203] F1[0.8967] at fold[5]
________________________________________________________________
Accuracy[0.9018] Recall[0.9022] F1[0.8743] at fold[6]
________________________________________________________________
Accuracy[0.8462] Recall[0.8551] F1[0.8540] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________
Accuracy

In [0]:
RunCatal('data/LOTO/USCHAD.npz')


Catal et al. 2015 data/LOTO/USCHAD.npz
Accuracy[0.8198] Recall[0.7916] F1[0.7913] at fold[1]
________________________________________________________________
Accuracy[0.8749] Recall[0.8400] F1[0.8407] at fold[2]
________________________________________________________________
Accuracy[0.9106] Recall[0.8891] F1[0.8849] at fold[3]
________________________________________________________________
Accuracy[0.8577] Recall[0.8290] F1[0.8325] at fold[4]
________________________________________________________________
Accuracy[0.8861] Recall[0.8496] F1[0.8465] at fold[5]
________________________________________________________________
Accuracy[0.8889] Recall[0.8410] F1[0.8421] at fold[6]
________________________________________________________________
Accuracy[0.8767] Recall[0.8462] F1[0.8478] at fold[7]
________________________________________________________________
Accuracy[0.8954] Recall[0.8419] F1[0.8478] at fold[8]
________________________________________________________________
Accuracy[

In [0]:
RunCatal('data/LOTO/UTD-MHAD1_1s.npz')


Catal et al. 2015 data/LOTO/UTD-MHAD1_1s.npz
Accuracy[0.5421] Recall[0.5305] F1[0.5287] at fold[1]
________________________________________________________________
Accuracy[0.4767] Recall[0.4757] F1[0.4746] at fold[2]
________________________________________________________________
Accuracy[0.4339] Recall[0.4430] F1[0.4339] at fold[3]
________________________________________________________________
Accuracy[0.5042] Recall[0.5038] F1[0.4998] at fold[4]
________________________________________________________________
Accuracy[0.4709] Recall[0.4649] F1[0.4630] at fold[5]
________________________________________________________________
Accuracy[0.4754] Recall[0.4764] F1[0.4740] at fold[6]
________________________________________________________________
Accuracy[0.4426] Recall[0.4437] F1[0.4449] at fold[7]
________________________________________________________________
Accuracy[0.5070] Recall[0.4990] F1[0.4978] at fold[8]
________________________________________________________________
Acc

In [0]:
RunCatal('data/LOTO/UTD-MHAD2_1s.npz')


Catal et al. 2015 data/LOTO/UTD-MHAD2_1s.npz
Accuracy[0.7692] Recall[0.7789] F1[0.7698] at fold[1]
________________________________________________________________
Accuracy[0.8281] Recall[0.8190] F1[0.8192] at fold[2]
________________________________________________________________
Accuracy[0.7453] Recall[0.7421] F1[0.7412] at fold[3]
________________________________________________________________
Accuracy[0.8700] Recall[0.8696] F1[0.8619] at fold[4]
________________________________________________________________
Accuracy[0.8053] Recall[0.8081] F1[0.7848] at fold[5]
________________________________________________________________
Accuracy[0.8364] Recall[0.8412] F1[0.8394] at fold[6]
________________________________________________________________
Accuracy[0.7589] Recall[0.7615] F1[0.7620] at fold[7]
________________________________________________________________
Accuracy[0.8300] Recall[0.8174] F1[0.8161] at fold[8]
________________________________________________________________
Acc

In [0]:
RunCatal('data/LOTO/WHARF.npz')

Catal et al. 2015 data/LOTO/WHARF.npz
Accuracy[0.6291] Recall[0.5566] F1[0.5665] at fold[1]
________________________________________________________________
Accuracy[0.6434] Recall[0.5348] F1[0.5400] at fold[2]
________________________________________________________________
Accuracy[0.5962] Recall[0.5337] F1[0.5199] at fold[3]
________________________________________________________________
Accuracy[0.6430] Recall[0.5756] F1[0.5811] at fold[4]
________________________________________________________________
Accuracy[0.6277] Recall[0.5368] F1[0.5188] at fold[5]
________________________________________________________________
Accuracy[0.6389] Recall[0.5611] F1[0.5647] at fold[6]
________________________________________________________________
Accuracy[0.6431] Recall[0.5625] F1[0.5623] at fold[7]
________________________________________________________________
Accuracy[0.6192] Recall[0.5226] F1[0.5246] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/LOTO/WISDM.npz')

Catal et al. 2015 data/LOTO/WISDM.npz
Accuracy[0.8973] Recall[0.8403] F1[0.8553] at fold[1]
________________________________________________________________
Accuracy[0.7904] Recall[0.6228] F1[0.6026] at fold[2]
________________________________________________________________
Accuracy[0.8453] Recall[0.7652] F1[0.7658] at fold[3]
________________________________________________________________
Accuracy[0.8481] Recall[0.7828] F1[0.7938] at fold[4]
________________________________________________________________
Accuracy[0.8333] Recall[0.7199] F1[0.7373] at fold[5]
________________________________________________________________
Accuracy[0.7562] Recall[0.7488] F1[0.7425] at fold[6]
________________________________________________________________
Accuracy[0.7487] Recall[0.7783] F1[0.7529] at fold[7]
________________________________________________________________
Accuracy[0.8369] Recall[0.8167] F1[0.8226] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/FNOW/MHEALTH.npz')


Catal et al. 2015 data/FNOW/MHEALTH.npz
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[1]
________________________________________________________________
Accuracy[0.9926] Recall[0.9931] F1[0.9930] at fold[2]
________________________________________________________________
Accuracy[0.9925] Recall[0.9931] F1[0.9930] at fold[3]
________________________________________________________________
Accuracy[0.9925] Recall[0.9931] F1[0.9930] at fold[4]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[5]
________________________________________________________________
Accuracy[0.9850] Recall[0.9861] F1[0.9860] at fold[6]
________________________________________________________________
Accuracy[0.9925] Recall[0.9931] F1[0.9930] at fold[7]
________________________________________________________________
Accuracy[1.0000] Recall[1.0000] F1[1.0000] at fold[8]
________________________________________________________________
Accuracy

In [0]:
RunCatal('data/FNOW/USCHAD.npz')


Catal et al. 2015 data/FNOW/USCHAD.npz
Accuracy[0.8841] Recall[0.8466] F1[0.8495] at fold[1]
________________________________________________________________
Accuracy[0.8878] Recall[0.8602] F1[0.8614] at fold[2]
________________________________________________________________
Accuracy[0.8653] Recall[0.8308] F1[0.8360] at fold[3]
________________________________________________________________
Accuracy[0.8929] Recall[0.8645] F1[0.8638] at fold[4]
________________________________________________________________
Accuracy[0.9087] Recall[0.8802] F1[0.8849] at fold[5]
________________________________________________________________
Accuracy[0.9008] Recall[0.8779] F1[0.8791] at fold[6]
________________________________________________________________
Accuracy[0.8688] Recall[0.8302] F1[0.8327] at fold[7]
________________________________________________________________
Accuracy[0.9002] Recall[0.8746] F1[0.8712] at fold[8]
________________________________________________________________
Accuracy[

In [0]:
RunCatal('data/FNOW/UTD-MHAD1_1s.npz')


Catal et al. 2015 data/FNOW/UTD-MHAD1_1s.npz
Accuracy[0.4413] Recall[0.4395] F1[0.4368] at fold[1]
________________________________________________________________
Accuracy[0.4554] Recall[0.4538] F1[0.4548] at fold[2]
________________________________________________________________
Accuracy[0.4524] Recall[0.4509] F1[0.4444] at fold[3]
________________________________________________________________
Accuracy[0.4423] Recall[0.4453] F1[0.4372] at fold[4]
________________________________________________________________
Accuracy[0.4515] Recall[0.4486] F1[0.4490] at fold[5]
________________________________________________________________
Accuracy[0.5049] Recall[0.5070] F1[0.5037] at fold[6]
________________________________________________________________
Accuracy[0.5198] Recall[0.5183] F1[0.5211] at fold[7]
________________________________________________________________
Accuracy[0.4824] Recall[0.4810] F1[0.4828] at fold[8]
________________________________________________________________
Acc

In [0]:
RunCatal('data/FNOW/UTD-MHAD2_1s.npz')


Catal et al. 2015 data/FNOW/UTD-MHAD2_1s.npz
Accuracy[0.7619] Recall[0.7516] F1[0.7506] at fold[1]
________________________________________________________________
Accuracy[0.8571] Recall[0.8605] F1[0.8632] at fold[2]
________________________________________________________________
Accuracy[0.8095] Recall[0.8123] F1[0.8036] at fold[3]
________________________________________________________________
Accuracy[0.8254] Recall[0.8132] F1[0.8242] at fold[4]
________________________________________________________________
Accuracy[0.8361] Recall[0.8319] F1[0.8363] at fold[5]
________________________________________________________________
Accuracy[0.8033] Recall[0.7995] F1[0.7997] at fold[6]
________________________________________________________________
Accuracy[0.7869] Recall[0.7835] F1[0.7750] at fold[7]
________________________________________________________________
Accuracy[0.8689] Recall[0.8725] F1[0.8685] at fold[8]
________________________________________________________________
Acc

In [0]:
RunCatal('data/FNOW/WHARF.npz')

Catal et al. 2015 data/FNOW/WHARF.npz
Accuracy[0.5545] Recall[0.4980] F1[0.5069] at fold[1]
________________________________________________________________
Accuracy[0.6484] Recall[0.5930] F1[0.5963] at fold[2]
________________________________________________________________
Accuracy[0.6073] Recall[0.5216] F1[0.5223] at fold[3]
________________________________________________________________
Accuracy[0.6313] Recall[0.5752] F1[0.5842] at fold[4]
________________________________________________________________
Accuracy[0.5899] Recall[0.5666] F1[0.5457] at fold[5]
________________________________________________________________
Accuracy[0.6103] Recall[0.5483] F1[0.5480] at fold[6]
________________________________________________________________
Accuracy[0.6321] Recall[0.5895] F1[0.5846] at fold[7]
________________________________________________________________
Accuracy[0.6066] Recall[0.5210] F1[0.5195] at fold[8]
________________________________________________________________
Accuracy[0

In [0]:
RunCatal('data/FNOW/WISDM.npz')

Catal et al. 2015 data/FNOW/WISDM.npz
Accuracy[0.8929] Recall[0.8335] F1[0.8502] at fold[1]
________________________________________________________________
Accuracy[0.8882] Recall[0.8366] F1[0.8544] at fold[2]
________________________________________________________________
Accuracy[0.9023] Recall[0.8506] F1[0.8669] at fold[3]
________________________________________________________________
Accuracy[0.8613] Recall[0.7945] F1[0.8086] at fold[4]
________________________________________________________________
Accuracy[0.9001] Recall[0.8419] F1[0.8620] at fold[5]
________________________________________________________________
Accuracy[0.8752] Recall[0.7971] F1[0.8197] at fold[6]
________________________________________________________________
Accuracy[0.8943] Recall[0.8367] F1[0.8520] at fold[7]
________________________________________________________________
Accuracy[0.8895] Recall[0.8320] F1[0.8513] at fold[8]
________________________________________________________________
Accuracy[0